In [4]:

from __future__ import print_function, division
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.nn import init
import torch.nn.functional as F
import functools
import numpy as np
import torchvision
from torchvision import transforms, datasets, models
import os
import time

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Training settings
batch_size = 50

In [6]:
#%%

# p = Augmentor.Pipeline()
# p.random_erasing(0.5,0.4)




train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop((32, 32), padding=4),   #left, top, right, bottom
#     p.torch_transform(),
#     p.sample(10000),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                               train=True,
                               transform=train_transform,
                               download=True)

test_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                              train=False,
                              transform=test_transform)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, # 64
                                           shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

dataloaders = {
    'train': train_loader,
    'test' : test_loader    
}
image_datasets = {
    'train': train_dataset,
    'test' : test_dataset
}
dataset_sizes = {
    'train': len(image_datasets['train']),
    'test' : len(image_datasets['test'])
}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print('train_size: ',dataset_sizes['train'],'\ntest_size : ',dataset_sizes['test'])
print(classes)
print('len class: ', len(classes))



Files already downloaded and verified
train_size:  50000 
test_size :  10000
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
len class:  10


In [50]:
import cv2
# # ADDING NOISE
noise = []
noise = np.random.rand(32, 32,3)
import pickle

# pickle_out = open("batch1","wb")
data = []
filenames = []
occludes = []
for n in range(2):
    inputs, labels = next(iter(dataloaders['train']))
    pickle_out = open("batch{}".format(n+1),"wb")
    data = []
    for m in range(batch_size):
        if(m<35):
            images_ = inputs[m].numpy()
            class_  = labels[m]
            images_ = np.transpose(images_, [1,2,0])
            
            r = images_[:,:,0]
            g = images_[:,:,1]
            b = images_[:,:,2]

            images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
            images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
            images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            data.append(images_)
            filename = '{}_{}.png'.format(classes[class_.numpy()],n*m+m)
            filenames.append(filename)
#         print(classes[class_.numpy()])
            plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_.numpy()],n*m+m), images_)
        
            occlude = 0
            occludes.append(occlude)
    #         plt.imshow(images_)
#         plt.show()
        else:
            images_ = inputs[m].numpy()
            class_  = labels[m]
            images_ = np.transpose(images_, [1,2,0])
            x_random = np.random.randint(6, 27, size=1, dtype='uint8')
            y_random = np.random.randint(6, 27, size=1, dtype='uint8')
            w_random = np.random.randint(8, 13, size=1, dtype='uint8')
            h_random = np.random.randint(8, 13, size=1, dtype='uint8')
            images_[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]=noise[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]

            r = images_[:,:,0]
            g = images_[:,:,1]
            b = images_[:,:,2]

            images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
            images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
            images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            data.append(images_)
            filename = '{}_{}.png'.format(classes[class_.numpy()],n*m+m)
            filenames.append(filename)
            
            plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_.numpy()],n*m+m), images_)
 
            occlude = 1
            occludes.append(occlude)
        
    pickle_cifar10 = {'labels': labels.numpy(),'data': data, 'filenames': filenames, 'occludes': occludes}
    pickle.dump(pickle_cifar10, pickle_out)
    pickle_out.close()
    data.clear()
    filenames.clear()
    occludes.clear()



Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fa7840632b0>>
Traceback (most recent call last):
  File "/home/lee/anaconda3/envs/pytorch1/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/lee/anaconda3/envs/pytorch1/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/lee/anaconda3/envs/pytorch1/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/lee/anaconda3/envs/pytorch1/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/lee/anaconda3/envs/pytorch1/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/lee/anaconda3/envs/pyt

In [7]:
def unpickle(file):
#     import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [51]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch1")
print(pickle_in)

{'labels': array([0, 5, 8, 5, 6, 1, 2, 5, 7, 5, 0, 4, 2, 3, 4, 0, 6, 9, 3, 8, 7, 8,
       3, 1, 7, 8, 7, 4, 0, 7, 7, 7, 9, 2, 8, 1, 4, 6, 2, 2, 2, 5, 3, 5,
       3, 3, 2, 5, 6, 2]), 'data': [array([[[0.68817204, 0.6813187 , 0.7657142 ],
        [0.87634414, 0.8791209 , 0.9485713 ],
        [0.83333343, 0.85714287, 0.8971429 ],
        ...,
        [0.97849464, 0.978022  , 0.9771429 ],
        [1.        , 0.96153855, 1.        ],
        [0.9892473 , 0.97252756, 0.9885714 ]],

       [[0.17204301, 0.1923077 , 0.3257143 ],
        [0.3978495 , 0.41758245, 0.56      ],
        [0.33333337, 0.3516484 , 0.47428572],
        ...,
        [0.97311836, 0.9835166 , 0.9714285 ],
        [0.9946237 , 0.96703297, 1.        ],
        [0.97849464, 0.978022  , 0.98285705]],

       [[0.02150538, 0.08791209, 0.19999999],
        [0.08602151, 0.14285715, 0.2685714 ],
        [0.10752688, 0.13186814, 0.2628571 ],
        ...,
        [0.97311836, 0.9835166 , 0.9714285 ],
        [1.        , 0.97802

In [52]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch2")
print(pickle_in)

{'labels': array([5, 9, 0, 6, 4, 3, 7, 0, 2, 6, 5, 8, 1, 9, 6, 6, 4, 3, 3, 0, 9, 6,
       8, 2, 0, 6, 5, 7, 9, 7, 0, 4, 9, 8, 2, 3, 5, 1, 1, 0, 7, 7, 1, 9,
       9, 3, 8, 7, 2, 2]), 'data': [array([[[0.8495935 , 0.91322315, 0.9590909 ],
        [0.8211382 , 0.87603307, 0.91818184],
        [0.8943089 , 0.91735536, 0.9409091 ],
        ...,
        [0.08536585, 0.05371901, 0.06818182],
        [0.4796748 , 0.4173554 , 0.47272727],
        [0.15447155, 0.08677686, 0.13181818]],

       [[0.8902439 , 0.9214876 , 0.9727273 ],
        [0.8211382 , 0.8512397 , 0.8909091 ],
        [0.7845529 , 0.7892562 , 0.8       ],
        ...,
        [0.10162602, 0.05785124, 0.09090909],
        [0.48373988, 0.4090909 , 0.47272727],
        [0.1585366 , 0.06611571, 0.13181818]],

       [[0.54471546, 0.54132235, 0.5590909 ],
        [0.43089432, 0.4338843 , 0.4318182 ],
        [0.39430895, 0.37603307, 0.3409091 ],
        ...,
        [0.11382115, 0.05785124, 0.09090909],
        [0.48373988, 0.40082

In [10]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/data/cifar-10-batches-py/data_batch_1")
print(pickle_in)

{b'batch_label': b'training batch 1 of 5', b'labels': [6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3, 6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4, 0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5, 6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2, 1, 1, 2, 1, 5, 9, 9, 0, 8, 4, 1, 1, 6, 3, 

In [11]:
pickle1_out = open("batch1_test","wb")

pickle.dump(pickle_in, pickle1_out)
pickle1_out.close()

In [15]:
dfd = unpickle("/home/lee/Research/11.project_tangqing/batch1_test")
print(dfd)

{b'batch_label': b'training batch 1 of 5', b'labels': [6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3, 6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4, 0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5, 6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2, 1, 1, 2, 1, 5, 9, 9, 0, 8, 4, 1, 1, 6, 3, 